In [0]:
spark.conf.set(
    "fs.azure.account.key.mystore1984.dfs.core.windows.net",
    "EEj+qfRk9tSnNApAaCMXikreuCmUhqg10Vgs1nUoH3uzL1iKy9WO9VyxQavv5giPl7lUtYEMcsGh+AStEI+fUg=="
)

In [0]:
dbutils.widgets.text('Date','')
input_value= dbutils.widgets.get('Date')

In [0]:
from pyspark.sql import SparkSession

# Start Spark session
spark = SparkSession.builder.getOrCreate()
df=spark.read\
    .option("header", True) \
    .option("inferSchema", True) \
    .csv('abfss://document@mystore1984.dfs.core.windows.net/customer*.csv')
df.show(2)

+------------+---+---+-------------+-------------+-------------+-----------+---------+----------------+--------------------+
|Customer ID |Sex|Age|Transaction 1|Transaction 2|Name/Location|Middle name|Last name|Transaction date|              Review|
+------------+---+---+-------------+-------------+-------------+-----------+---------+----------------+--------------------+
|         200|  M|M24|        $1000|         $100| James/Canada|      David|   Harris|      2025-01-10|The iPhone's came...|
|         201|  F|F27|         $500|          $50|     Sham/USA|          P| Anderson|      2025-01-10|The iPhone's Face...|
+------------+---+---+-------------+-------------+-------------+-----------+---------+----------------+--------------------+
only showing top 2 rows



In [0]:
from pyspark.sql.functions import regexp_replace, col, split, concat_ws,regexp_extract,lit

In [0]:
df_cleaned = df.withColumn('Transaction 1',regexp_replace(col('Transaction 1'), r'\$', ' '))\
       .withColumn('Transaction 2',regexp_replace(col('Transaction 2'), r'\$', ' '))\
       .withColumn('Age',regexp_replace(col('Age'), r'(M|F)', ' '))\
       .withColumn('Transaction 1',col('Transaction 1').cast('int'))\
       .withColumn('Transaction 2',col('Transaction 2').cast('int'))\
       .withColumn('Age',col('Age').cast('int'))\
       .withColumn('Name',split(col('Name/Location'), '/').getItem(0))\
       .withColumn('Location',split(col('Name/Location'), '/').getItem(1))\
       .withColumn('Full name',concat_ws(' ',col('Name'), col('Middle name'), col('Last name')))\
       .withColumn('category',regexp_extract(col('Review'), r'(iPhone|Samsung Galaxy|Motorola)',1))\
       .drop('Name/Location','Middle name','Last name','Name','Review')\
       .withColumnRenamed('Transaction 1', 'Transaction1') \
       .withColumnRenamed('Transaction 2', 'Transaction2')\
       .withColumnRenamed('Customer ID ', 'CustomerID')\
       .withColumn("Date",lit(input_value))
       

In [0]:
df_cleaned.show()

+----------+----+----+------------+------------+----------------+--------+--------------------+--------------+----+
|CustomerID| Sex| Age|Transaction1|Transaction2|Transaction date|Location|           Full name|      category|Date|
+----------+----+----+------------+------------+----------------+--------+--------------------+--------------+----+
|       200|   M|  24|        1000|         100|      2025-01-10|  Canada|  James David Harris|        iPhone|    |
|       201|   F|  27|         500|          50|      2025-01-10|     USA|     Sham P Anderson|        iPhone|    |
|       202|   M|  30|         700|          70|      2025-01-10|     USA|William Thomas Ha...|Samsung Galaxy|    |
|       203|   F|  33|         300|          30|      2025-02-21|     USA|           yuih Luis|      Motorola|    |
|       204|null|4567|         500|          50|      2025-02-21|  Canada|        Rabin Turner|        iPhone|    |
|       205|   M|null|         700|         700|      2025-02-21|     US

In [0]:
from pyspark.sql.functions import col

mode_Sex = (df.groupBy("Sex").count().orderBy(col("count").desc()).first()[0])
median_Age = df_cleaned.approxQuantile("Age", [0.5],0)[0]
df_cleaned= df_cleaned.fillna({"Age": median_Age, "Sex": mode_Sex})

In [0]:
from pyspark.sql.functions import col, when

# List of numeric columns to clip
numeric_cols = ["Transaction1", "Transaction2", "Age"]

for c in numeric_cols:
    # Compute Q1 and Q3
    q1, q3 = df_cleaned.approxQuantile(c, [0.25, 0.75], 0.0)
    IQR = q3 - q1
    upper_limit = q3 + 1.5 * IQR
    lower_limit = q1 - 1.5 * IQR
    
    # Clip the column
    df_cleaned = df_cleaned.withColumn(
        c,
        when(col(c) > upper_limit, upper_limit)
        .when(col(c) < lower_limit, lower_limit)
        .otherwise(col(c))
    )

In [0]:
from pyspark.sql.functions import expr
df_cleaned = df_cleaned.withColumn("total", expr("(Transaction1+ Transaction2)"))\
                       .withColumn("level", expr("case when total> 1000 then 'High' when total> 500 then 'Medium' else 'low' end"))\
                       .withColumn("Transaction date",expr("date_format(`Transaction date`, 'yyyy-MM-dd')"))\
                       .withColumn("Transaction Month",expr("date_format(`Transaction date`, 'yyyy-MM')"))

In [0]:
df_cleaned=df_cleaned.withColumnRenamed('Transaction date', 'TransactionDate')\
                     .withColumnRenamed('Transaction Month', 'TransactionMonth')\
                     .withColumnRenamed('Full name', 'Name')

In [0]:
df_cleaned.show()

+----------+---+----+------------+------------+---------------+--------+--------------------+--------------+----+------+------+----------------+
|CustomerID|Sex| Age|Transaction1|Transaction2|TransactionDate|Location|                Name|      category|Date| total| level|TransactionMonth|
+----------+---+----+------------+------------+---------------+--------+--------------------+--------------+----+------+------+----------------+
|       200|  M|24.0|      1000.0|       100.0|     2025-01-10|  Canada|  James David Harris|        iPhone|    |1100.0|  High|         2025-01|
|       201|  F|27.0|       500.0|        50.0|     2025-01-10|     USA|     Sham P Anderson|        iPhone|    | 550.0|Medium|         2025-01|
|       202|  M|30.0|       700.0|        70.0|     2025-01-10|     USA|William Thomas Ha...|Samsung Galaxy|    | 770.0|Medium|         2025-01|
|       203|  F|33.0|       300.0|        30.0|     2025-02-21|     USA|           yuih Luis|      Motorola|    | 330.0|   low|   

In [0]:
%sql
create database dataanalysis

In [0]:
%sql
use dataanalysis

In [0]:
df_cleaned.write.format("delta").mode("overwrite").saveAsTable("data_table")

In [0]:
%sql
select * from data_table

CustomerID,Sex,Age,Transaction1,Transaction2,TransactionDate,Location,Name,category,Date,total,level,TransactionMonth
200,M,24.0,1000.0,100.0,2025-01-10,Canada,James David Harris,iPhone,,1100.0,High,2025-01
201,F,27.0,500.0,50.0,2025-01-10,USA,Sham P Anderson,iPhone,,550.0,Medium,2025-01
202,M,30.0,700.0,70.0,2025-01-10,USA,William Thomas Harris,Samsung Galaxy,,770.0,Medium,2025-01
203,F,33.0,300.0,30.0,2025-02-21,USA,yuih Luis,Motorola,,330.0,low,2025-02
204,M,60.0,500.0,50.0,2025-02-21,Canada,Rabin Turner,iPhone,,550.0,Medium,2025-02
205,M,33.0,700.0,700.0,2025-02-21,USA,suren Kumar sarkar,Samsung Galaxy,,1400.0,High,2025-02
206,M,42.0,300.0,30.0,2025-02-21,Canada,jina palu,Motorola,,330.0,low,2025-02
207,F,45.0,1000.0,1000.0,2025-03-05,Canada,Brian S Harris,iPhone,,2000.0,High,2025-03
208,M,32.0,2050.0,1675.0,2025-03-05,Canada,Akib,Samsung Galaxy,,3725.0,High,2025-03
209,F,33.0,300.0,300.0,2025-03-05,USA,Richard Anderson,Motorola,,600.0,Medium,2025-03


In [0]:
%sql
select category,
       sum(total)as total_cost
       from data_table
       group by category
       order by total_cost

category,total_cost
Motorola,1260.0
iPhone,4200.0
Samsung Galaxy,5895.0


Databricks visualization. Run in Databricks to view.